- BOW(Back of Words)에는 문서에서 단어의 사용 어부만 표시하는 방법과 단어 수를 세어 표시하는 방법, 단어가 문서에 나타난 수를 반여해 보정하는 방법 등이 있다.


## 카운트 기반 문서 표현의 개념
- 카운트 기반 문서 표현은 문서의 의미를 반영해 만드는 과정이라 할 수 있다.
- 일반적으로 글을 이해할 때 앞에서부터 순서대로 단어들을 읽어가는 것처럼, 텍스트와 관련한 딥러닝 기법 역시 같은 방식으로 학습니다.
- 예를 들어 컴퓨터가 책에서 내용을 빠르게 읽는 방법은 책에 나오는 단어의 비도를 세는 것이다.
- 텍스트 마이닝을 하기 위해서는 이러한 텍스트들을 수치 형태로 변환해야 한다.
- 각 수치는 그 텍스트의 특성(feature)을 표현한다.
- 카운트 기반의 문서표현에서는 텍스트의 특성을 단어로 표현하고, 특성이 갖는 값은 그 단어가 텍스트에서 나타나는 횟수로 표현한다.


- 문제는 이렇게 하기 위해서는 모든 문서의 특성이 같아야 서로 비교할 수 있는데, 딕셔너리로 표현하면 특성이 제각각이어서 다른 방식이 필요한다.
- 이를 위해 컴퓨터는 동일한 단어들로 특성을 표현한다.
- 즉 전체 말뭉치에 한 번이라도 사용된 단어는 문서에 없더라도 특성을 포함하고 빈도를 0으로 준다.
- 따라서 카운트 기반의 문서 표현은 개별 분서가 아닌 말뭉치기 그 대상이다.


## 희소 벡터(Sparse Vector)
- 하나의 문서에서 사용한 단어에 비해 사용하지 않은 단어가 훨씬 많을 수밖에 없다.
- 문서에서 사용되니 않은 대부분의 특성값은 0이 된다.
- 결국 문서를 표현하기 위해 너무나 많은 특성을 사용해야 하고, 그 특성 중 극히 일부만 값을 갖는다.
- 이와 같이 대부분의 값이 0인 특성 벡터를 희소 벡터라하는데, 저장공간과 연산 측변에서 매우 비효율적이므로 이를 해결할 수 있는 나름의 방법이 필요하다.


In [245]:
#NLTK 데이터 사용 
# movie_reviews 데이터는 연구를 위해 수집된 2000개의 영화 리뷰와 
# 이 내용이 긍정적인지 부정적인지를 나타내는 감성 값이 함께 제공 

import nltk as nltk 

nltk.download('punkt_tab')
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\3호실-09\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\3호실-09\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [246]:
print('review count: ', len(movie_reviews.fileids()))
print('sample of file ids: ', movie_reviews.fileids()[:10])

review count:  2000
sample of file ids:  ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']


In [247]:
fileid = movie_reviews.fileids()[0] # 첫 번째 문서의 id를 반환
print('#id of the first review: ', fileid)


# 첫번째 문서의 내용을 200자까지만 출력
print('#first review content:\n', movie_reviews.raw(fileid)[:200])


# 첫번째 문서를 sentence tokenize한 결과 중 앞 두 문장
print('#sentence tokenization result: ', movie_reviews.sents(fileid)[:2])


# 첫번째 문서를 word tokenize한 결과 중 앞 20개 단어
print('#word tokenization result: ', movie_reviews.words(fileid)[:20])


#id of the first review:  neg/cv000_29416.txt
#first review content:
 plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
w
#sentence tokenization result:  [['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.']]
#word tokenization result:  ['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an']


In [248]:
# 각 문서에 대한 토큰화 결과 리스트 만들기 (빈도 계산을 위한 리스트화)
# fileid()를 이용해 모든 문서의 id를 가져오고 각 id들에 대해 words()로 토큰화 결과를 가져와 리스트를 만든다. 

documents = [list(movie_reviews.words(fileid)) for fileid in movie_reviews.fileids()]
print(documents[0][:50])


['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch']


In [249]:
# 특성 집함을 만들기 위해 딕셔너리를 이용해 단어별 말뭉치 전체에서 빈도를 계산
# 빈도가 높은 단어부터 정렬한 후 빈도수 상위 10개 단어 출력

word_count = {}
for text in documents:
  for word in text:
    word_count[word] = word_count.get(word, 0) + 1 # word_count.get(word, 0)를 통해 현재 단어가 word_count에 존재하면 그 값을 가져오고, 없으면 기본값 0을 반환.


sorted_features = sorted(word_count, key=word_count.get, reverse=True)
for word in sorted_features[:10]:
  print(f'count of "{word}": {word_count[word]}', end=", ")


count of ",": 77717, count of "the": 76529, count of ".": 65876, count of "a": 38106, count of "and": 35576, count of "of": 34123, count of "to": 31937, count of "'": 30585, count of "is": 25195, count of "in": 21822, 

In [250]:
# 위 코드 결과를 보면 the a 같이 의미 없는 단어의 빈도가 높다 (언어적 패딩 / 불용어 )
# 이를 해결하기 위해 정규표현식으로 다시 토큰화 한다. 
# 먼저 raw()를 이용해 원문을 가져와서 documents를 만들고, 이에 대해 토큰화 한다 

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords # 일반적으로 분석 대상이 아닌 단어들
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\3호실-09\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [251]:
# [\w]: 알파벳 대소문자(A-Z, a-z), 숫자(0-9), 그리고 밑줄(_)에 해당하는 문자들을 나타낸다. 알파벳에서 대소문자만 구분하여 추출
# {3,}: 바로 앞의 패턴([\w])이 3번 이상 반복되는 문자열을 매칭한다.
tokenizer = RegexpTokenizer("[\w]{3,}") # 정규 표현식으로 토크나이저 정의
english_stops = set(stopwords.words('english')) # 영어 불용어 호출

# words() 대신 raw()로 원문을 가져옴
documents = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]


# stopwords의 적용과 토큰화를 동시 수행
tokens = [[token for token in tokenizer.tokenize(doc) if token not in english_stops] for doc in documents]


word_count = {}
for text in tokens:
  for word in text:
    word_count[word] = word_count.get(word, 0) + 1


sorted_features = sorted(word_count, key=word_count.get, reverse=True)


print('num of features: ', len(sorted_features))


for word in sorted_features[:10]:
  print(f"count of '{word}': {word_count[word]}", end=",")


<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
C:\Users\3호실-09\AppData\Local\Temp\ipykernel_5280\149581058.py:3: SyntaxWarning: invalid escape sequence '\w'
  tokenizer = RegexpTokenizer("[\w]{3,}") # 정규 표현식으로 토크나이저 정의


num of features:  39208
count of 'film': 9517,count of 'one': 5852,count of 'movie': 5771,count of 'like': 3690,count of 'even': 2565,count of 'good': 2411,count of 'time': 2411,count of 'story': 2169,count of 'would': 2109,count of 'much': 2049,

In [255]:
# 빈도가 높은 상위 1000개 단어만 추출해 features 구성 
word_features = sorted_features[:1000]

In [256]:
# 특성 벡터, 즉 카운트 벡터로 변환
# 예: 특성 집합 예제(word_features_ex): ['one', 'two', 'teen', 'couples', 'solo']
# 예: 주어진 문서의 토큰화 결과: ['two', 'two', 'couples']
# 변환된 벡터 결과: [0, 2, 0, 1, 0]
# 위 예시문서에서 'two'가 2회 'couples'가 1회 사용됐으므로, 전체 특성 리스트에 맞워봤을 때 둘째와 넷째 값이 각각 2와 1이 된다.


def document_features(document, word_features):
  word_count = {}
  for word in document:
    word_count[word] = word_count.get(word, 0) + 1


  features = []
  # word_features의 단어에 대해 계산된 빈도수를 feature에 추가
  for word in word_features:
    features.append(word_count.get(word, 0)) # 빈도가 없는 단어는 0을 입력
  return features


word_features_ex = ['one', 'two', 'teen', 'couples', 'solo']
document_ex = ['two', 'two', 'couples']
print(document_features(document_ex, word_features_ex))


[0, 2, 0, 1, 0]


In [257]:
# 위 함수를 이용해 전체 리뷰 집합에 대해 그대로 적용 
# 추출된 특성의 수와 첫밴째 리뷰 문서에 대한 특성 집합 중 앞 20개만 출력 
# 벡터에 있는 값에 대해 어떤 단어의 빈도인지를 알기 위해 순서를 맞춰 word_features의 단어와 함께 출력 

feature_sets = [document_features(d, word_features) for d in tokens]

# 첫번째 feature set 의 내용을 앞 20개만 word_features의 단어와 함께 출력 

for i in range(20):
    print(f'({word_features[i]},{feature_sets[0][i]})')

(film,6)
(one,3)
(movie,6)
(like,3)
(even,3)
(good,2)
(time,0)
(story,0)
(would,1)
(much,0)
(character,2)
(also,1)
(get,3)
(two,2)
(well,1)
(characters,1)
(first,0)
(see,2)
(way,3)
(make,5)


## 사이킷런을 통한 카운트 벡터 생성
- 사이킷런의 텍스트 관련 라이브러리는 사용하기가 훨씬 편하다.
- 텍스트나 이미지로부터 특성을 추출하는 도구로 sklearn.feature_extraction 모듈이 있다.
- 가장 기본적인 도구는 CountVectorizer 클래스다
- 참조: https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


- 주요 메서드
 - fit(raw_documents): 문서 집합에 대해 토큰화를 수행하고 특성집합 생성
 - transoform(raw_documents): fit()에서 생성한 특성 집합을 이용해 인수로 주어진 문서 집합에 대해 카우트 벡터로 변환
 - fit_transform(raw_documents): 인수로 주어진 문서집합에 대해 fit과 transform을 동시에 수행
 - get_feature_names_out(): 특성 집합에 있는 특성의 이름, 즉 단어를 순서대로 반환


- 한글의 경우 KoNLPy로 형태소 분석을 수행해야 하므로 별도의 토크나이저를 사용해야 한다.


In [258]:
# 데이터 로딩 
reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]

# 전처리 없이 바로 분석을 시작할 때는 max_features에 빈도가 높은 단어부터 사용할 단어 수를 지정함으로써
# 원하는 벡터를 구성할 수 있다.
# 사이킷런에서는 단어들이 알파벳 순으로 정렬된다는 차이가 있다.

from sklearn.feature_extraction.text import CountVectorizer

# 1. 모든 매개변수에 기본값을 적용하는 경우 
# cv = CountVectorizer()

# 2 특성 집합을 지정하는 경우 
cv = CountVectorizer(vocabulary=word_features)

# 3 특성 집합을 지정하지 않고, 최대 특성의 수를 지정하는 경우 
# cv = CountVectorizer(max_features=1000)

print(cv)

CountVectorizer(vocabulary=['film', 'one', 'movie', 'like', 'even', 'good',
                            'time', 'story', 'would', 'much', 'character',
                            'also', 'get', 'two', 'well', 'characters', 'first',
                            'see', 'way', 'make', 'life', 'really', 'films',
                            'plot', 'little', 'people', 'could', 'scene', 'man',
                            'bad', ...])


In [259]:
# 객체 생성 후 fit_transform()을 사용해 특성 집합을 생성하고 카운트 벡터를 생성 
reviews_cv = cv.fit_transform(reviews)
print(reviews_cv)

# 왼쪽은 튜플 좌표를 나타내고, 오른쪽은 숫자의 빈도수를 나타낸다. 
# (0,5), (0,7)이 빠져있는데, 이는 카운트 벡터가 매우 희소해서 제외 

  (0, 0)	6
  (0, 1)	3
  (0, 2)	6
  (0, 3)	3
  (0, 4)	3
  (0, 5)	2
  (0, 8)	1
  (0, 10)	2
  (0, 11)	1
  (0, 12)	3
  (0, 13)	2
  (0, 14)	1
  (0, 15)	1
  (0, 17)	2
  (0, 18)	3
  (0, 19)	5
  (0, 20)	1
  (0, 21)	2
  (0, 22)	2
  (0, 23)	1
  (0, 24)	2
  (0, 25)	1
  (0, 29)	2
  (0, 32)	1
  (0, 33)	2
  :	:
  (1999, 730)	1
  (1999, 735)	20
  (1999, 744)	1
  (1999, 751)	2
  (1999, 756)	1
  (1999, 766)	1
  (1999, 786)	1
  (1999, 791)	1
  (1999, 800)	1
  (1999, 805)	1
  (1999, 822)	1
  (1999, 838)	1
  (1999, 840)	1
  (1999, 859)	2
  (1999, 878)	1
  (1999, 880)	1
  (1999, 883)	1
  (1999, 899)	1
  (1999, 904)	2
  (1999, 906)	1
  (1999, 911)	1
  (1999, 914)	1
  (1999, 934)	2
  (1999, 940)	1
  (1999, 956)	1


In [260]:
print(cv.get_feature_names_out()[:20])
print('-'*50)
print(word_features[:20])

['film' 'one' 'movie' 'like' 'even' 'good' 'time' 'story' 'would' 'much'
 'character' 'also' 'get' 'two' 'well' 'characters' 'first' 'see' 'way'
 'make']
--------------------------------------------------
['film', 'one', 'movie', 'like', 'even', 'good', 'time', 'story', 'would', 'much', 'character', 'also', 'get', 'two', 'well', 'characters', 'first', 'see', 'way', 'make']


In [261]:
reviews_cv
# 2000 * 1000 행렬이고, 그안에 254,135개의 값이 있다. 
# 이들을 제외한 나머지 값은 0이다 
# 비율을 계산해 보면 254135/(2000*1000) * 100 = 12.65%가 된다. 
# 빈도가 1000개 보다 많이 지정했으면 더 희소한 행렬이 된다. 

<2000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 254135 stored elements in Compressed Sparse Row format>

In [262]:
# 가공해서 모델에 사용하기 위해서는 array 형식의 리스트가 되어야 한다 
print(feature_sets[0][:20])
print(reviews_cv.toarray()[0, :20])

[6, 3, 6, 3, 3, 2, 0, 0, 1, 0, 2, 1, 3, 2, 1, 1, 0, 2, 3, 5]
[6 3 6 3 3 2 0 0 1 0 2 1 3 2 1 1 0 2 3 5]


## 코사인 유사도


- 추출한 벡터는 머신러닝 기법을 적용하기 위한 입력으로 사용된다
- 이 벡터는 또한 문서 간의 유사도를 측정하는 데에도 사용된다.
- 대표적인 유사도 척도는 코사인 유사도다.


In [268]:
# 사이킷런에서는 유사도를 측정할 수 있는 클래스를 제공한다.
# 이를 확인하기 위해 NLTK 영화 리뷰 문서들에 대한 유사도를 측정한다.
# 첫번째 리뷰의 뒷부분 절발을 잘라서 대상 문서를 만든다.
# 유사도 측정이 제대로 된다면 새로 만든 문서와 가장 유사한 문서로 첫 번째 리뷰가 나올 것이다.
# 코사인 유사도 계산을 위해서는 카운트 벡터가 필요하기 때문에 새로 만든 문서에 대해서도
# 카운트 벡터로 변환한 후 유사도를 계산한다.
# 이 때 사이킷런의 cosine_similarity 함수는 벡터 하나씩 유사도를 계산하지 않고
# 다수 벡터와 다수 벡터 간 유사도를 한 번에 계산하고 그 결과를 행렬로 반환한다.
# m개의 벡터와 n개의 벡터 간의 유사도를 한꺼번에 계산했다면 결과는 (m, n)의 2차원 행렬로 변환된다.
# 여기 예에서는 대상 문서가 하나이고 이를 전체 리뷰와 비교하므로 결과는 (1, 2000)의 행렬을 반환한다.



In [269]:
# 첫번쨰 리뷰의 문자수를 확인하고 뒤 절반을 가져오기 위해 중심점을 찾는다 
start = len(reviews[0]) // 2
print(start)

2021


In [274]:
# 중심점으로 부터 뒤 절반을 가져와 비교할 문서 생성 
source = reviews[0][-start:]

from sklearn.metrics.pairwise import cosine_similarity



# 코사인유사도는 카운트 벡터에 대해 계산하므로 벡터써야함
# transform 은 반드시 리스트나 행렬 형태로 입력을 요구한다.

source_cv = cv.transform([source])

# 행렬의 크기를 확인, 문서가 하나이므로 (1, 1000)
print('대상 특성 행렬크기:' , source_cv.shape)

# 변환된 카운트 벡터와 기존 값들과의 유사도 계산 
sim_result = cosine_similarity(source_cv, reviews_cv)

print(' 유사도 계산결과를 역순으로 정렬:' , sorted(sim_result[0], reverse=True)[:10])
print(' 유사도 계산결과를 정순으로 정렬:' , sim_result[0][:10])

대상 특성 행렬크기: (1, 1000)
 유사도 계산결과를 역순으로 정렬: [0.8367205630128807, 0.43784070017231075, 0.4080451370075411, 0.4063802391472214, 0.4060219836225451, 0.39965783997760135, 0.3994228018440035, 0.39566661804603703, 0.3938902773761537, 0.3911637170821695]
 유사도 계산결과를 정순으로 정렬: [0.83672056 0.29618526 0.3011952  0.20001903 0.21937004 0.22626503
 0.21742292 0.23618092 0.21460684 0.34614362]


In [275]:
# 분석 리뷰가 첫 번째 리뷰인지 확인, 넘파이의 argmax함수를 사용 
import numpy as np 
print('가장 유사한 리뷰의 인덱스' , np.argmax(sim_result[0]))


가장 유사한 리뷰의 인덱스 0


## TF-IDF(Term requency - Inverse Document Frequency)


- 카운트 벡터에서 빈도는 일종의 가중치로 작용한다.
- 즉 빈도가 높을수록 중요한 단어로 취급되는 경향이 있다.
- 하지만 만일 어떤 단어가 모든 문서에 나타난다면 그 단어는 중요하다고 판단하기 어렵디ㅏ.
- 이것을 정도로 바꿧 말하면, 단어가 더 많은 문서에서 나타날수록 그 단어는 별로 중요하지 않게 된다.
- 이러한 의미를 카운트 벡터에 반영한 것이 TF-IDF다.
- 이는 카운트 대신 단어의 빈도에 그 단어가 출현한 문서 수의 역수를 곱한다. 즉, 단어의 빈도를 그 단어가 나타난 문서의 수로 나눈다.
- 이렇게 하면 단어가 나타난 문서의 수가 클수록 이 값은 작아져서 목표한 평가 결과를 얻을 수 있다.


In [278]:
# 사이킷 런은 텍스트로 부터 바로 TF-IDF 행렬을 생성하는 TfidfTransformer 클래스를 제공한다 
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
transformer

reviews_tfidf =transformer.fit_transform(reviews_cv)

#TF-IDF 행렬의 모양(2000)과 카운트 행렬(1000) 의 모양 확인 
print(reviews_tfidf.shape)

# 첫번쨰 리뷰의 카운터 벡터 중 앞의 20개 값 출력 
print(reviews_cv[0].toarray()[0][:20])

# 첫 번쨰 리뷰의 TF-IDF 벡터 중 앞의 20개 값 출력 
print(reviews_tfidf[0].toarray()[0][:20])

(2000, 1000)
[6 3 6 3 3 2 0 0 1 0 2 1 3 2 1 1 0 2 3 5]
[0.13556199 0.06700076 0.14998642 0.0772298  0.08608998 0.0609124
 0.         0.         0.03126552 0.         0.06575035 0.03242315
 0.09567082 0.06518293 0.03225625 0.0345017  0.         0.06863314
 0.10042383 0.16727495]


In [281]:
# word_features를 사용한 유사도 계산 

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(vocabulary=word_features)

reviews_tf = tf.fit_transform(reviews)

# 코사인 유사도는 카운트 벡터에 대해 계산하므로 비교 문자 단어를 벡터로 변환 
source_tf = tf.transform([source])

# 변환된 카운트 벡터와 기존 값들과의 유사도 계산 
sim_result_tf = cosine_similarity(source_tf, reviews_tf)
print('가장 유사한 리뷰 인덱스: ', np.argmax(sim_result_tf[0]))


가장 유사한 리뷰 인덱스:  0
